In [107]:
import numpy as np
import pandas as pd

In [108]:
data= pd.read_csv("owid-covid-data-most-recent.csv")
data = data.dropna(subset=['total_deaths'])
filtered_columns = [col for col in data.columns if 'smoothed' not in col]
data = data[filtered_columns]
data = data.loc[df['date'] > '2020-03-01']

In [109]:
missing_counts = data.groupby('location').apply(lambda x: x.isnull().sum())
total_missing_per_country = missing_counts.sum(axis=1)
sorted_countries = total_missing_per_country.sort_values(ascending=True)
sorted_countries.head(10)

location
Tuvalu           1170
Estonia          1629
Italy            1724
Czechia          1986
Ireland          2017
Israel           2021
France           2162
Belgium          2334
Slovakia         2345
United States    2382
dtype: int64

In [110]:
missingness_prop = sorted_countries / len(data)
sorted_countries = sorted_countries.to_frame(name='total_missing').join(missingness_prop.rename('missingness_prop'))
sorted_countries.head(10)

,total_missing,missingness_prop
location,,
Tuvalu,1170,0.026401
Estonia,1629,0.036759
Italy,1724,0.038902
Czechia,1986,0.044815
Ireland,2017,0.045514
Israel,2021,0.045604
France,2162,0.048786
Belgium,2334,0.052667
Slovakia,2345,0.052915


In [111]:
top_countries = sorted_countries.head(10).index
data_smaller = data[data['location'].isin(top_countries)]
data_smaller

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,stringency_index,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
24362,BEL,Europe,Belgium,2020-03-15,1362.0,1046.0,9.0,9.0,116.850,89.740,0.772,0.772,2.38,53.0,4.547,263.0,22.564,NaN,NaN,221.0,18.960,14412.0,1269.0,1.241,0.109,0.095,10.5,tests performed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.93,375.564,41.8,18.571,12.849,42658.576,0.2,114.898,4.29,25.1,31.4,NaN,5.64,81.63,0.931,11655923.0,-1995.09990,-7.40,-4.51,-172.56088
24369,BEL,Europe,Belgium,2020-03-22,4473.0,3111.0,149.0,140.0,383.753,266.903,12.783,12.011,2.13,326.0,27.969,1661.0,142.503,NaN,NaN,1499.0,128.604,31478.0,1414.0,2.711,0.122,0.222,4.5,tests performed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.48,375.564,41.8,18.571,12.849,42658.576,0.2,114.898,4.29,25.1,31.4,NaN,5.64,81.63,0.931,11655923.0,-1758.19980,-6.00,10.22,-152.07083
24376,BEL,Europe,Belgium,2020-03-29,12031.0,7558.0,751.0,602.0,1032.179,648.426,64.431,51.648,1.82,961.0,82.447,4536.0,389.158,NaN,NaN,3499.0,300.191,58619.0,2419.0,5.048,0.208,0.327,3.1,tests performed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.48,375.564,41.8,18.571,12.849,42658.576,0.2,114.898,4.29,25.1,31.4,NaN,5.64,81.63,0.931,11655923.0,-781.89984,-2.48,43.94,-67.62836
24383,BEL,Europe,Belgium,2020-04-05,21665.0,9634.0,2108.0,1357.0,1858.712,826.533,180.852,116.421,1.46,1267.0,108.700,5620.0,482.158,NaN,NaN,3571.0,306.368,97045.0,3828.0,8.358,0.330,0.282,3.5,tests performed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.48,375.564,41.8,18.571,12.849,42658.576,0.2,114.898,4.29,25.1,31.4,NaN,5.64,81.63,0.931,11655923.0,1124.60010,3.35,90.16,97.26931
24390,BEL,Europe,Belgium,2020-04-12,32874.0,11209.0,4046.0,1938.0,2820.369,961.657,347.120,166.267,1.21,1253.0,107.499,5542.0,475.466,NaN,NaN,2830.0,242.795,150562.0,7287.0,12.967,0.628,0.219,4.6,tests performed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.48,375.564,41.8,18.571,12.849,42658.576,0.2,114.898,4.29,25.1,31.4,NaN,5.64,81.63,0.931,11655923.0,3326.50000,9.32,105.35,287.71680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261344,USA,North America,United States,2023-12-10,103436829.0,NaN,1157683.0,NaN,305763.910,NaN,3422.163,NaN,NaN,2425.0,7.168,19794.0,58.512,NaN,NaN,23814.0,70.395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.608,38.3,15.413,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.926,338289856.0,NaN,NaN,NaN,NaN
261351,USA,North America,United States,2023-12-17,103436829.0,NaN,1159133.0,NaN,305763.910,NaN,3426.449,NaN,NaN,2478.0,7.325,21106.0,62.390,NaN,NaN,25601.0,75.678,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.608,38.3,15.413,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.926,338289856.0,NaN,NaN,NaN,NaN
261358,USA,North America,United States,2023-12-24,103436829.0,NaN,1160396.0,NaN,305763.910,NaN,3430.

In [112]:
missing_values = data_smaller.isnull().sum()
missingness_prop = missing_values / len(data_smaller)
column_missingness = missing_values.to_frame(name='total_missing').join(missingness_prop.rename('missingness_prop'))

In [113]:
def highlight(val):
    color = 'red' if val > 0.5 else 'black'
    return f'color: {color}'

In [114]:
column_missingness = column_missingness.style.applymap(highlight, subset=['missingness_prop'])
column_missingness

,total_missing,missingness_prop
iso_code,0,0.000000
continent,0,0.000000
location,0,0.000000
date,0,0.000000
total_cases,0,0.000000
new_cases,34,0.018630
total_deaths,0,0.000000
new_deaths,34,0.018630
total_cases_per_million,0,0.000000
new_cases_per_million,34,0.018630


In [115]:
corr = pd.DataFrame(data_smaller.corrwith(df["total_deaths"]))
corr = corr.rename(columns={0: 'Correlation with total_deaths'})
corr.style.applymap(highlight)

,Correlation with total_deaths
total_cases,0.959357
new_cases,0.485359
total_deaths,1.000000
new_deaths,0.499900
total_cases_per_million,0.076757
new_cases_per_million,0.004435
total_deaths_per_million,0.319139
new_deaths_per_million,0.008876
reproduction_rate,-0.050977
icu_patients,0.422576
